## INSTALANDO E IMPORTANDO A NEWSAPI

In [1]:
!pip install --upgrade pip
!pip install newsapi-python
!pip install pandas
!pip install python-dotenv

In [2]:
from newsapi import NewsApiClient
from datetime import datetime
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os

## CHAVE DA API

In [3]:
load_dotenv()
api_key = os.getenv('API_KEY')
newsapi = NewsApiClient(api_key=api_key)


## EXTRAÇÃO DE DADOS

In [4]:
def extracao_dados():
    criterios = ['genoma', 'sequenciamento de dna', 'terapias geneticas personalizadas', 'medicina personalizada', 'Síndrome de Down', 'Fibrose cística', 'Hemofilia']
    artigos = []
    
    for criterio in criterios:
        resultado = newsapi.get_everything(q=criterio,
                                            language='pt',
                                            sort_by='relevancy'
                                            )
        if resultado['status']=='ok':
            
            for artigo in resultado['articles']:
                artigo['criterio_relevancia']=criterio               
            artigos.extend(resultado['articles'])
            
    print(f'Quantidade de artigos achados: {len(artigos)}')
    with open("dados_brutos.json", "w") as arquivo_json:
        json.dump(artigos, arquivo_json)
   

## TRANSFORMAÇÃO DOS DADOS

In [5]:
def transformacao_dados():

    artigos = pd.read_json('dados_brutos.json')

    fontes = []
    for i in range(len(artigos)):
        fontes.append(artigos['source'][i]['name'])

    artigos['fonte'] = fontes
    del artigos['source']

        
    colunas_renomeadas = {
        'author':'autor',
        'title':'titulo',
        'description':'descricao',
        'publishedAt':'publicado_em',
        'content':'conteudo'    
    }
    colunas_selecao = ['titulo','descricao','conteudo','autor','fonte','url','publicado_em','criterio_relevancia']

  
    df = pd.DataFrame(artigos)
    df = df.loc[df.title!='[Removed]']
    df = df.fillna('')
    df = df.rename(columns=colunas_renomeadas)
    df = df[colunas_selecao]
    df['publicado_em'] = pd.to_datetime(df['publicado_em'], format='%Y-%m-%dT%H:%M:%SZ')

#LER O ARQUIVO JA SALVO E CONCATENA COM O NOVO DATAFRAME
    try:
        df_antigo = pd.read_json('dados_tratados_em_json.json')
        df = pd.concat([df, df_antigo], axis=0)
        
        df = df.drop_duplicates(subset=['url']).reset_index(drop=True)
                
    
    except Exception as e:
        df = df.drop_duplicates(subset=['url']).reset_index(drop=True)
        print('Primeira leitura')

    return df
    

## Load do DataFrame final

In [6]:
def load(df):
    df.to_json('dados_tratados_em_json.json')

## ETL

In [7]:
def etl():
    extracao_dados()
    df = transformacao_dados()
    load(df)
    return df


## Cria uma função para contar um tempo e permitir um processamento a cada periodo de tempo definido

In [8]:
def timer(tempo_proc):
    print("inicializa contagem")
    for t in range(1, tempo_proc+1):
        print(t)
        time.sleep(1)
    print("finaliza contagem")

## Faz a chamada definitiva do processamento a cada 5 segundos

In [9]:
while 1==1:
    timer(5)
    df = etl()
    df.head()


inicializa contagem
1
2
3
4
5
finaliza contagem
Quantidade de artigos achados: 126
Primeira leitura
inicializa contagem
1
2
3
4
5
finaliza contagem
Quantidade de artigos achados: 126
Estou no Try
inicializa contagem
1
2
3
4
5
finaliza contagem
Quantidade de artigos achados: 126
Estou no Try
inicializa contagem
1


KeyboardInterrupt: 

In [ ]:
df = etl()
df.head()

Quantidade de artigos achados: 122
Primeira leitura


,titulo,descricao,conteudo,autor,fonte,url,publicado_em,criterio_relevancia
0,Árvore “congelada” no tempo por milhões de ano...,"Conhecida como ""fóssil vivo"", a árvore pratica...","Cientistas da AustrÃ¡lia, Estados Unidos e ItÃ...",Alessandro Di Lorenzo,Olhardigital.com.br,https://olhardigital.com.br/2023/09/15/ciencia...,2023-09-15 17:06:28,genoma
1,Estudo indica onde fica o lar original da huma...,Pesquisadores criaram uma rede de quase 27 mil...,A histÃ³ria da humanidade Ã© marcada pelas mig...,Alessandro Di Lorenzo,Olhardigital.com.br,https://olhardigital.com.br/2023/09/14/ciencia...,2023-09-14 20:40:37,genoma
2,Novo vírus é descoberto nas profundezas da Fos...,"Bacteriófago, um tipo de vírus que infecta e s...",Segundo dados da AdministraÃ§Ã£o Nacional OceÃ...,Alessandro Di Lorenzo,Olhardigital.com.br,https://olhardigital.com.br/2023/09/21/ciencia...,2023-09-21 20:26:34,genoma
3,Miniórgãos descobertos em mamíferos podem gera...,Pesquisadores do Instituto Federal Suíço de Te...,Pesquisadores do Instituto Federal Suíço de Te...,Jorge Marin,Tecmundo.com.br,https://www.tecmundo.com.br/ciencia/272244-min...,2023-10-02 20:30:04,genoma
4,Pesquisadores analisam a complexidade da doenç...,Equipe utilizou mais de duas milhões de célula...,Investigando os potenciais tratamentos de Alzh...,Alisson Santos,Olhardigital.com.br,https://olhardigital.com.br/2023/09/29/medicin...,2023-09-29 09:00:00,genoma


## Algumas outras transformações

In [ ]:
# def noticias_por_data(df):

    
#     df = df.groupby(by=[df.publicado_em.dt.date]).agg(quantidade=('titulo', 'count')).sort_index()
#     df = df.reset_index(drop=False)

#     print("Pesquisa de noticias por data realizada com sucesso\n")    
#     print(df)

# def noticias_por_fonte_autor(df):

   
#     df = df.groupby(by=['fonte','autor']).agg(quantidade=('titulo', 'count'))
    
#     df = df.reset_index(drop=False)

 

#     print("Pesquisa de noticias por fonte e autor realizada com sucesso\n")
#     print(df)

# def aparicoes_palavras(df):

    
#     df = df.groupby(by=[df.publicado_em.dt.date, 'criterio_relevancia']).agg(quantidade=('titulo', 'count')).sort_index()

    
#     df = df.reset_index(drop=False)

    

#     print("Pesquisa de noticias por aparicao realizada com sucesso\n")
#     print(df)

In [ ]:

# noticias_por_data = noticias_por_data(df)
# noticias_por_data

In [ ]:
# noticias_por_fonte_autor = noticias_por_fonte_autor(df)
# noticias_por_fonte_autor

In [ ]:
# aparicoes_palavras = aparicoes_palavras(df)
# aparicoes_palavras